# 第8章 Attention

本章的目标是在代码层面理解 Attention的结构，然后将其应用于实际问题，体验它的奇妙效果。

## 8.1 Attention的结构

介绍进一步强化 seq2seq 的**注意力机制**（attention mechanism，简称
Attention）。基于 Attention 机制，seq2seq 可以像我们人类一样，将“注
意力”集中在必要的信息上。此外，使用 Attention 可以解决当前 seq2seq
面临的问题。

### 8.1.1 seq2seq存在的问题

seq2seq 中使用编码器对时序数据进行编码，然后将编码信息传递给解
码器。此时，编码器的输出是固定长度的向量。实际上，这个“固定长度”
存在很大问题。因为固定长度的向量意味着，无论输入语句的长度如何（无
论多长），都会被转换为长度相同的向量。

![](../images/图8-1.无论输入语句多长，编码器都将其塞入固定长度的向量中.PNG)
图8-1.无论输入语句多长，编码器都将其塞入固定长度的向量中

### 8.1.2 编码器的改进

![](../images/图8-2.使用编码器各个时刻（各个单词）的LSTM层的隐藏状态（这里表示为hs）.PNG)
图8-2.使用编码器各个时刻（各个单词）的LSTM层的隐藏状态（这里表示为hs）

如图 8-2 所示，使用各个时刻（各个单词）的隐藏状态向量，可以获得和输入的单词数相同数量的向量。

各个时刻的 LSTM 层的隐藏状态都充满了什么信息呢？
有一点可以确定的是，各个时刻的隐藏状态中包含了大量当前时刻的输入单词的信息。

![](../images/图8-3.编码器的输出hs具有和单词数相同数量的向量，各个向量中蕴含了各个单词对应的信息.PNG)
图8-3.编码器的输出hs具有和单词数相同数量的向量，各个向量中蕴含了各个单词对应的信息

这里我们所做的改进只是将编码器的全部时刻的隐藏状态取出来而已。通过这个小改动，编码器可以根据输入语句的长
度，成比例地编码信息。

### 8.1.3 编码器的改进1

编码器整体输出各个单词对应的 LSTM 层的隐藏状态向量 hs。然后，
这个 hs 被传递给解码器，以进行时间序列的转换（图 8-4）。

![](../images/图8-4.编码器和解码器的关系.PNG)
图8-4.编码器和解码器的关系

![](../images/图8-5.上一章的解码器的层结构（学习时）.PNG)
图8-5.上一章的解码器的层结构（学习时）


在机器翻译的历史中，很多研究都利用“猫 =cat”这样的单词对应
关系的知识。这样的表示单词（或者词组）对应关系的信息称为**对齐**
（alignment）。到目前为止，对齐主要是手工完成的，而我们将要介
绍的 Attention 技术则成功地将对齐思想自动引入到了 seq2seq 中。
这也是从“手工操作”到“机械自动化”的演变。

是找出与“翻译目标词”有对应关系的“翻译源词”的信息，然后利用这个信息进行翻译。
我们的目标是仅关注必要的信息，并根据该信息进行时序转换。这个机制称为 Attention。

![](../images/图8-6.改进后的解码器的层结构.PNG)
图8-6.改进后的解码器的层结构

可否将“选择”这一操作换成可微分的运算呢？实际上，解决这个问题
的思路很简单。这个思路就是，与其“单选”，不如“全选”。

![](../images/图8-7.对各个单词计算表示重要度的权重（后文介绍如何计算）.PNG)
图8-7.对各个单词计算表示重要度的权重（后文介绍如何计算）

![](../images/图8-8.通过计算加权和，可以得到上下文向量.PNG)
图8-8.通过计算加权和，可以得到上下文向量

上下文向量 $c$ 中包含了当前时刻进行变换（翻译）所需的信息。更确
切地说，模型要从数据中学习出这种能力。

In [2]:
import numpy as np

T, H = 5, 4
hs = np.random.randn(T, H)
a = np.array([0.8, 0.1, 0.03, 0.05, 0.02])

ar = a.reshape(5, 1).repeat(4, axis=1)
print(ar.shape)  # (5, 4)

t = hs * ar
print(t.shape)  # (5, 4)

c = np.sum(t, axis=0)
print(c.shape)  # (4,)

(5, 4)
(5, 4)
(4,)


![](../images/图8-9.通过reshape()和repeat()方法从a生成ar（变量的形状显示在其右侧）.PNG)
图8-9.通过reshape()和repeat()方法从a生成ar（变量的形状显示在其右侧）

![](../images/图8-10.NumPy的广播.PNG)
图8-10.NumPy的广播

In [3]:
# 批处理版的加权和的实现
N, T, H = 10, 5, 4
hs = np.random.randn(N, T, H)
a = np.random.randn(N, T)
ar = a.reshape(N, T, 1).repeat(H, axis=2)
# ar = a.reshape(N, T, 1) # 使用广播
t = hs * ar
print(t.shape) # (10, 5, 4)

c = np.sum(t, axis=1)
print(c.shape) # (10, 4)

(10, 5, 4)
(10, 4)


![](../images/图8-11.加权和的计算图.PNG)
图8-11.加权和的计算图

将图8-11 的计算图实现为层，这里称之为 Weight Sum 层，其实现如下所示（[ch08/attention_layer.py](../ch08/attention_layer.py)）。

### 8.1.4 解码器改建2

![](../images/图8-12.解码器第1个LSTM层的隐藏状态向量.PNG)
图8-12.解码器第1个LSTM层的隐藏状态向量

向量 $a = (a_1, a_2, \cdots , a_n)$ 和向量 $b = (b_1, b_2, \cdots , b_n)$ 的内积为：
$$
    a \cdot b = a_1b_1 + a_2b_2 + \cdots + a_nb_n \tag{8.1}
$$

式 (8.1) 的含义是两个向量在多大程度上指向同一方向，因此使用内积作为两个向量的“相似度”是非常自然的选择。

![](../images/图8-13.基于内积计算hs的各行与h的相似度（内积用dot节点表示.PNG)
图8-13.基于内积计算hs的各行与h的相似度（内积用dot节点表示）

![](../images/图8-14.基于Softmax的正规化.PNG)
图8-14.基于Softmax的正规化

In [5]:
import sys
sys.path.append('..')
from common.layers import Softmax
import numpy as np

N, T, H = 10, 5, 4
hs = np.random.randn(N, T, H)
h = np.random.randn(N, H)
hr = h.reshape(N, 1, H).repeat(T, axis=1)
# hr = h.reshape(N, 1, H) # 广播

t = hs * hr
print(t.shape) # (10, 5, 4)

s = np.sum(t, axis=2)
print(s.shape) # (10, 5)

softmax = Softmax()
a = softmax.forward(s)
print(a.shape) # (10, 5)

(10, 5, 4)
(10, 5)
(10, 5)


![](../images/图8-15.计算各个单词权重的计算图.PNG)
图8-15.计算各个单词权重的计算图

将这个计算图表示的处理实现为 AttentionWeight 类（[ch08/attention_layer.py](../ch08/attention_layer.py)）。

### 8.1.5 解码器改进3

![](../images/图8-16.计算上下文向量的计算图.PNG)
图8-16.计算上下文向量的计算图

Attention Weight 层关注编码器输出的各个单词向量 hs，并计算各个单词的权重 a；
然后，Weight Sum 层计算 a 和 hs 的加权和，并输出上下文向量 c。
我们将进行这一系列计算的层称为 Attention 层（图 8-17）。

![](../images/图8-17.将左边的计算图整合为Attention层.PNG)
图8-17.将左边的计算图整合为Attention层

Attention 层的实现（[ch08/attention_layer.py](../ch08/attention_layer.py)）。

![](../images/图8-18.具有Attention层的解码器的层结构.PNG)
图8-18.具有Attention层的解码器的层结构

如图 8-18 所示，编码器的输出 hs 被输入到各个时刻的 Attention 层。
另外，这里将 LSTM 层的隐藏状态向量输入 Affine 层。

![](../images/图8-19.上一章的解码器（左图）和带Attention的解码器（右图）的比较：选出从LSTM层到Affine层的部分.PNG)
图8-19.上一章的解码器（左图）和带Attention的解码器（右图）的比较：选出从LSTM层到Affine层的部分

如图 8-19 所示，我们向上一章的解码器“添加”基于 Attention 层
的上下文向量信息。因此，除了将原先的 LSTM 层的隐藏状态向量传给
Affine 层之外，追加输入 Attention 层的上下文向量。

![](../images/图8-20.将多个Attention层整体实现为Time%20Attention层.PNG)
图8-20.将多个Attention层整体实现为Time Attention层

由图 8-20 可知，Time Attention 层只是组合了多个 Attention 层，其实现如下所示（[ch08/attention_layer.py](../ch08/attention_layer.py#86)）

## 8.2 带Attention的seq2seq的实现

上一章实现了3个类（Encoder、Decoder 和 seq2seq）一样，
这里我们也分别实现 3 个类（AttentionEncoder、AttentionDecoder 和 AttentionSeq2seq）。

### 8.2.1 编码器的实现

AttentionEncoder 类的实现如下所示（[ch08/attention_seq2seq.py](../ch08/attention_seq2seq.py)）

### 8.2.2 解码器的实现

![](../images/图8-21.解码器的层结构.PNG)
图8-21.解码器的层结构

### 8.2.3 seq2seq的实现

## 8.3 Attention的评价

### 8.3.1 日期格式转换问题

![](../images/图8-22.日期格式转换的例子.PNG)
图8-22.日期格式转换的例子

![](../images/图8-23.用于日期格式转换的学习数据：空格显示为灰点.PNG)
图8-23.用于日期格式转换的学习数据：空格显示为灰点

### 8.3.2 带Attention的 seq2seq的学习

在日期转换用的数据集上进行 AttentionSeq2seq 的学习，学习用的代码如下所示（[ch08/train.py](../ch08/train.py)）

![](../images/图8-25.正确率的演变.PNG)
图8-25.正确率的演变

![](../images/图8-26.与其他模型的比较.PNG)
图8-26.与其他模型的比较：图中的baseline是上一章中的简单的seq2seq，peeky是使用“偷窥”技术改进过的seq2seq（所有的模型都反转了输入语句）

在这次的实验中，就最终精度来看，Attention 和 Peeky 取得了差
不多的结果。但是，随着时序数据变长、变复杂，除了学习速度之外，
Attention 在精度上也会变得更有优势。

### 8.3.3 Attention的可视化

实现中，Time Attention 层中的成员变量 attention_weights 保
存了各个时刻的 Attention 权重，据此可以将输入语句和输出语句的各个单词
的对应关系绘制成一张二维地图。

[ch08/visualize_attention.py](../ch08/visualize_attention.py)

![](../images/图8-27.使用学习好的模型，对进行时序转换时的Attention权重进行可视化。.PNG)
图8-27.使用学习好的模型，对进行时序转换时的Attention权重进行可视化。

![](../images/图8-28.Attention权重的例子.PNG)
图8-28.Attention权重的例子

我们没有办法理解神经网络内部进行了什么工作（基于何种逻辑工作），
而 Attention 赋予了模型“人类可以理解的结构和意义”。在上面的
例子中，通过 Attention，我们看到了单词和单词之间的关联性。由此，
我们可以判断模型的工作逻辑是否符合人类的逻辑。

## 8.4 关于Attention的其他话题

### 8.4.1 双向RNN

![](../images/图8-29.基于LSTM层输出hs.PNG)
图8-29.基于LSTM层输出hs

可以让 LSTM 从两个方向进行处理，这就是名为双向 LSTM 的技术

![](../images/图8-30.基于双向LSTM进行编码的例子（这里简化了LSTM层）.PNG)
图8-30.基于双向LSTM进行编码的例子（这里简化了LSTM层）

双向 LSTM 在之前的 LSTM 层上添加了一个反方向
处理的 LSTM 层。然后，拼接各个时刻的两个 LSTM 层的隐藏状态，将
其作为最后的隐藏状态向量（除了拼接之外，也可以“求和”或者“取平
均”等）

通过这样的双向处理，各个单词对应的隐藏状态向量可以从左右两个方
向聚集信息。这样一来，这些向量就编码了更均衡的信息。

TimeBiLSTM类中有双向 LSTM 的实现（[common/time_layers.py](../common/time_layers.py)）

### 8.4.2 Attention层的使用

![](../images/图8-31.上一节之前使用的带Attention的seq2seq的层结构.PNG)
图8-31.上一节之前使用的带Attention的seq2seq的层结构

Affine 层使用了上下文向量:

![](../images/图8-32.Attention层的其他使用示例.PNG)
图8-32.Attention层的其他使用示例

### 8.4.3 seq2seq的深层化和skip connection

![](../images/图8-33.使用了3层LSTM层的带Attention的seq2seq.PNG)
图8-33.使用了3层LSTM层的带Attention的seq2seq

在加深层时使用到的另一个重要技巧是**残差连接**（skip connection，也称为 residual connection 或 shortcut）。

![](../images/图8-34.LSTM层中的skip%20connection的例子.PNG)
图8-34.LSTM层中的skip connection的例子

如图 8-34 所示，所谓残差连接，就是指“跨层连接”。

因为加法在反向传播时“按原样”传播梯度，所以残差
连接中的梯度可以不受任何影响地传播到前一个层。这样一来，即便加深了
层，梯度也能正常传播，而不会发生梯度消失（或者梯度爆炸），学习可以
顺利进行。

在时间方向上，RNN 层的反向传播会出现梯度消失或梯度爆炸的问
题。梯度消失可以通过 LSTM、GRU 等 Gated RNN 应对，梯度爆
炸可以通过梯度裁剪应对。而对于深度方向上的梯度消失，这里介
绍的残差连接很有效。

## 8.5 Attention的应用

### 8.5.1 GNMT（神经机器翻译（Neural Machine Translation））

机器翻译的历史：
* 基于规则的翻译
* 基于用例的翻译
* 基于统计的翻译
* **神经机器翻译**（Neural Machine Translation）

神经机器翻译这个术语是出于与之前的基于统计的翻译进行对比而使用的，
现在已经成为使用了 seq2seq 的机器翻译的统称。


**GNMT**（Google Neural Machine Translation，谷歌神经机器翻译系统）

![](../images/图8-35.GNMT的层结构.PNG)
图8-35.GNMT的层结构

GNMT 和本章实现的带 Attention 的 seq2seq 一样，由编码器、解码
器和 Attention 构成。不过，与我们的简单模型不同，这里可以看到许多为
了提高翻译精度而做的改进，比如 LSTM 层的多层化、双向 LSTM（仅编
码器的第 1 层）和 skip connection 等。另外，为了提高学习速度，还进行
了多个 GPU 上的分布式学习。

![](../images/图8-36.GNMT的精度评价：纵轴是人按照0~6对翻译质量进行的评价.PNG)
图8-36.GNMT的精度评价：纵轴是人按照0~6对翻译质量进行的评价

### 8.5.2 Transformer

RNN 也有缺点，比如并行处理的问题。
RNN 需要基于上一个时刻的计算结果逐步进行计算，因此（基本）不
可能在时间方向上并行计算 RNN。

现在关于去除 RNN 的研究（可以并行计算的RNN 的研究）很活跃，其中一个著名的模型是 Transformer 模型。
Transformer 是在“Attention is all you need”这篇论文中提出来的方法。
如论文标题所示，Transformer 不用 RNN，而用 Attention 进行处理。

除了 Transformer 之外，还有多个研究致力于去除 RNN，比如用
卷积层（Convolution 层）代替 RNN 的研究。

Transformer 是基于 Attention 构成的，其中使用了 **Self-Attention** 技
巧，这一点很重要。Self-Attention 直译为“自己对自己的 Attention”，也
就是说，这是以一个时序数据为对象的 Attention，旨在观察一个时序数
据中每个元素与其他元素的关系。用 Time Attention 层来说明的话，Self Attention 如图 8-37 所示

![](../images/图8-37.左图是常规的Attention，右图是Self-Attention.PNG)
图8-37.左图是常规的Attention，右图是Self-Attention

![](../images/TheTransformer-model-architecture.PNG)
The Transformer - model architecture.

![](../images/图8-38.Transformer的层结构.PNG)
图8-38.Transformer的层结构

Self-Attention 的两个输入中输入的是同一个时序数据。像这样，可以求得一个时序数据内各个元素
之间的对应关系。

Transformer 中用 Attention 代替了 RNN。
编码器和解码器两者都使用了 Self-Attention。
Feed Forward层表示前馈神经网络（在时间方向上独立的网络）。具体而言，使用具有一
个隐藏层、激活函数为 ReLU 的全连接的神经网络。另外，图中的 Nx 表
示灰色背景包围的元素被堆叠了 N 次。

图 8-38 显示的是简化了的 Transformer。实际上，除了这个架构
外，Skip Connection、Layer Normalization等技巧也会被用到。
其他常见的技巧还有，（并行）使用多个 Attention、编码时序数据
的位置信息（Positional Encoding，位置编码）等。


使用 Transformer 可以控制计算量，充分利用 GPU 并行计算带来的好处。

![](../images/图8-39.使用基准翻译数据WMT，评价英法翻译的精度.PNG)
图8-39.使用基准翻译数据WMT，评价英法翻译的精度。

### 8.5.3 NTM（Neural Turing Machine，神经图灵机）

基于外部存储装置的扩展

RNN 和 LSTM 能够使用内部状态来存储时序数据，但是它们的内
部状态长度固定，能塞入其中的信息量有限。因此，可以考虑在
RNN 的外部配置存储装置（内存），适当地记录必要信息。

![](../images/图8-40.NTM的概念图.PNG)
图8-40.NTM的概念图

![](../images/图8-41.NTM的层结构：新出现了Write%20Head层和Read%20Head层，它们进行内存读写.PNG)
图8-41.NTM的层结构：新出现了Write Head层和Read Head层，它们进行内存读写

## 8.6 小结

## 本章所学的内容

* 在翻译、语音识别等将一个时序数据转换为另一个时序数据的任务中，时序数据之间常常存在对应关系
* Attention 从数据中学习两个时序数据之间的对应关系
* Attention 使用向量内积（方法之一）计算向量之间的相似度，并输出这个相似度的加权和向量
* 因为 Attention 中使用的运算是可微分的，所以可以基于误差反向传播法进行学习
* 通过将 Attention 计算出的权重（概率）可视化，可以观察输入与输出之间的对应关系
* 在基于外部存储装置扩展神经网络的研究示例中，Attention 被用来读写内存